In [ ]:
from dataclasses import dataclass, fields, field
from abc import ABCMeta, abstractmethod
from pyspark.sql.functions import col
from datetime import datetime
from typing import TypeVar
import pyspark.pandas as ps
import dlt

### AdjudicatedSvcs Fields

#### Sourcing the Fields ####
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.adjudicatedsvcs ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsclaims.adjudicatedsvcs* 


| Source Field | Type Comment |
| :--- | :--- |
| cgbeforeadjcoinsuranceamt | Coinsurance Amount:  The 3 Fields (Copay, Deductible, Coinsurance). They are based upon GROSS AMOUNT APPROVED |
| cgbeforeadjcopayamt | Copay Amount:  The 3 Fields (Copay, Deductible, Coinsurance). They are based upon GROSS AMOUNT APPROVED |
| cgbeforeadjdeductibleamt | Deductible Amount:  The 3 Fields (Copay, Deductible, Coinsurance). They are based upon GROSS AMOUNT APPROVED |
| clmnum | Claim Number (MONTHLY SEE FIELD NAME = CLAIM_NO) (MONTHLY SEE FIELD NAME = CLAIM_NO) (Pointer to ^EF0-CLAIMS) |
| enctrsvcsvcnum | Encounter Service Service Number (MONTHLY SEE FIELD NAME = ENC_SER_NO) (2 of 2 col; Pointer to ^ENX-ENCOUNTER_SERVICES) |
| prof | Professional (MONTHLY SEE FIELD NAME = SERV_PHY_ID) (Pointer to ^DI1D-PHYSICIANS) |
| cgbeforeadjamtprepaid | Before Adjustment Amount Prepaid |


#### Accessing the Data ####
| AdjudicatedSvcs Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| AdjudicatedSvcsData( ).adjudicatedsvcs_base_table( ) | AdjudicatedSvcsData( ).create_adjudicatedsvcs_data_df( ) | AdjudicatedSvcsData( ).view_adjudicatedsvcs_metadata( ) |


#### Development Comments ####
Summary
- AdjudicatedSvcs medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the AdjudicatedSvcsMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalAdjudicatedServices(metaclass = ABCMeta):
    @property
    @abstractmethod
    def adjudicatedsvcs_dataframe(self):
        ...
    
    @adjudicatedsvcs_dataframe.setter
    @abstractmethod
    def adjudicatedsvcs_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_adjudicatedsvcs_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_adjudicatedsvcs_metadata():
        ...

    @staticmethod
    @abstractmethod
    def adjudicatedsvcs_base_table(self):
        ...


@dataclass(frozen=True)
class AdjudicatedSvcsMetadata:
    claim_no: int = field(metadata={"claim_no": "Claim Number (MONTHLY SEE FIELD NAME = CLAIM_NO) (MONTHLY SEE FIELD NAME = CLAIM_NO) (Pointer to ^EF0-CLAIMS)"})

    enc_ser_no: int = field(metadata={"enc_ser_no": "Service line Encounter Serial Number.  Each unique calc_cpt4_cd receives a service line number."})

    phy_id: int = field(metadata={"phy_id": "Internal physician identification number"})

    pre_paid_amt: int = field(metadata={"pre_paid_amt": "Health plan pre-paid amount after Accounts Payable has processed the claim. Pre_Paid_Amt is sourced directly from CACHE and calculated at the remit level. These uncommon payments are usually for Physical Therapy or Chiropractic care and are generally very small dollar amounts.Source Field = CLAIM_SER_PYMT_IS.PRE_PAID_AMT except for FQHC (Federally Qualified Health Center) lines.  * PURCH_TP_NM = MEDICAID * SERV_CAT_LEVEL1_ID in (1, 3) * CLAIM_FORM_TP_CD = HCFA 1500 * FQHC_FLG = YFQHC Service Year <= 20191. Join to Federal Medicaid file by Service Year, Procedure & Modifier to get FQHC_RVU and FQHC_CONV_FAC.2. When there was not a match, join only by Service Year & Procedure.3. Recalculate PRE_PAID_AMT = UNIT_QTY * FQHC_RVU * FQHC_CONV_FACFQHC Service Year > 20191. Join to Federal Medicaid file by Service Year, Procedure & Modifier to get FQHC_RATE.2. When there was not a match, join only by Service Year & Procedure.3. Recalculate PRE_PAID_AMT = UNIT_QTY * FQHC_RATE* Not allocated to the service line level at extract.  Exactly as it come out of the claim system."})
    

class AdjudicatedSvcsData(MedicalAdjudicatedServices, AdjudicatedSvcsMetadata):
    ADJUDICATEDSVCS_DF = spark.read.table('cleansed.adminsystemsclaims.adjudicatedsvcs')
    def __init__(self):
        self.__adjudicatedsvcs_df = None

    @property
    def adjudicatedsvcs_dataframe(self):
        return self.__adjudicatedsvcs_df
    
    @adjudicatedsvcs_dataframe.setter
    def adjudicatedsvcs_dataframe(self, dataframe):
        adjudicatedsvcs_df = dataframe.select(
            col("CLMNUM")\
                .cast(str(self.__dataclass_fields__["claim_no"].type.__name__))\
                    .alias("claim_no", metadata={"claim_no": self.__dataclass_fields__["claim_no"].metadata.get("claim_no")}),
            col("ENCTRSVCSVCNUM")\
                .cast(str(self.__dataclass_fields__["enc_ser_no"].type.__name__))\
                    .alias("enc_ser_no", metadata={"enc_ser_no": self.__dataclass_fields__["enc_ser_no"].metadata.get("enc_ser_no")}),
            col("PROF")\
                .cast(str(self.__dataclass_fields__["phy_id"].type.__name__))\
                    .alias("phy_id", metadata={"phy_id": self.__dataclass_fields__["phy_id"].metadata.get("phy_id")}),
            col("CGBEFOREADJAMTPREPAID")\
                .cast(str(self.__dataclass_fields__["pre_paid_amt"].type.__name__))\
                    .alias("pre_paid_amt", metadata={"pre_paid_amt": self.__dataclass_fields__["pre_paid_amt"].metadata.get("pre_paid_amt")})
            )
        self.__adjudicatedsvcs_df = adjudicatedsvcs_df

    def create_adjudicatedsvcs_data_df(self):
        self.adjudicatedsvcs_dataframe = self.ADJUDICATEDSVCS_DF
        _adjudicatedsvcs_dataframe = self.adjudicatedsvcs_dataframe
        _adjudicatedsvcs_dataframe.display()    
        return _adjudicatedsvcs_dataframe
    
    @staticmethod
    def view_adjudicatedsvcs_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(AdjudicatedSvcsMetadata)
            ]
        adjudicatedsvcs_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        adjudicatedsvcs_metadata_df.display()

    @staticmethod
    def adjudicatedsvcs_base_table():
        AdjudicatedSvcsData.ADJUDICATEDSVCS_DF.display()
    
    def __repr__(self):
        return f'{self.__adjudicatedsvcs_df.display()}'

### BenefitPkg Fields

#### Sourcing the Fields ####
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.benefitpkg ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsmember.benefitpkg* 


| Source Field | Type Comment |
| :--- | :--- |
| pkgcode | The Benefit Package Code attached to the members coverage.  Please note package code changes to a Hospice code when member enters Hospice care. |
| pkgnum | 'ben_pkg_id' comes from enctrfrppackages.pkgnum |


#### Accessing the Data ####
| BenefitPkg Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| BenefitPkgData( ).benefitpkg_base_table( ) | BenefitPkgData( ).create_benefitpkg_data_df( ) | BenefitPkgData( ).view_benefitpkg_metadata( ) |


#### Development Comments ####
Summary
- BenefitPkg medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the BenefitPkgMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalBenefitPackage(metaclass = ABCMeta):
    @property
    @abstractmethod
    def benefitpkg_dataframe(self):
        ...
    
    @benefitpkg_dataframe.setter
    @abstractmethod
    def benefitpkg_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_benefitpkg_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_benefitpkg_metadata():
        ...

    @staticmethod
    @abstractmethod
    def benefitpkg_base_table(self):
        ...


@dataclass(frozen=True)
class BenefitPkgMetadata:
    string = TypeVar("string", bound=str)

    ben_pkg_cd: string = field(metadata={"ben_pkg_cd": "The Benefit Package Code attached to the members coverage.  Please note package code changes to a Hospice code when member enters Hospice care"})

    package_no: int = field(metadata={"package_no": "ben_pkg_id comes from enctrfrppackages.pkgnum"})
    

class BenefitPkgData(MedicalBenefitPackage, BenefitPkgMetadata):
    BENEFITPKG_DF = spark.read.table('cleansed.adminsystemsmember.benefitpkg')

    def __init__(self):
        self.__benefitpkg_df = None

    @property
    def benefitpkg_dataframe(self):
        return self.__benefitpkg_df
    
    @benefitpkg_dataframe.setter
    def benefitpkg_dataframe(self, dataframe):
        benefitpkg_df = dataframe.select(
            col("PkgCode")\
                .cast(str(self.__dataclass_fields__["ben_pkg_cd"].type.__name__))\
                    .alias("ben_pkg_cd", metadata={"ben_pkg_cd": self.__dataclass_fields__["ben_pkg_cd"].metadata.get("ben_pkg_cd")}),
            col("PkgNum")\
                .cast(str(self.__dataclass_fields__["package_no"].type.__name__))\
                    .alias("package_no", metadata={"package_no": self.__dataclass_fields__["package_no"].metadata.get("package_no")})
            )
        self.__benefitpkg_df = benefitpkg_df

    def create_benefitpkg_data_df(self):
        self.benefitpkg_dataframe = self.BENEFITPKG_DF
        _benefitpkg_dataframe = self.benefitpkg_dataframe
        _benefitpkg_dataframe.display()    
        return _benefitpkg_dataframe
    
    @staticmethod
    def view_benefitpkg_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(BenefitPkgMetadata)
            ]
        benefitpkg_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        benefitpkg_metadata_df.display()

    @staticmethod
    def benefitpkg_base_table():
        BenefitPkgData.BENEFITPKG_DF.display()
    
    def __repr__(self):
        return f'{self.__benefitpkg_df.display()}'

### Claims Fields

#### Sourcing the Fields
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.claims ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsclaims.claims* 


| Source Field | Type Comment |
| :--- | :--- |
| acceptancestatus | Acceptance Status (MONTHLY SEE FIELD NAME = ACCEPT_STAT_CD) (Pointer to ^DI3a-ACCEPTANCE_STATUSES) |
| amtallowedredbyothins | Amount Allowed Reduction By Other Insurance |
| clmnum | Claim Number (MONTHLY SEE FIELD NAME = CLAIM_NO) |
| enctrnum | Encounter number generated by the production system. (MONTHLY SEE FIELD NAME = ENC_NO) Pointer to ^ENV-ENCOUNTERS) |
| frpnum | FRP Number (MONTHLY SEE FIELD NAME = FRP_NO) |


#### Accessing the Data
| Claims Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| ClaimsData( ).claims_base_table( ) | ClaimsData( ).create_claims_data_df( ) | ClaimsData( ).view_claims_metadata( ) |


#### Development Comments
Summary
- Claims medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the ClaimsMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalClaims(metaclass = ABCMeta):
    @property
    @abstractmethod
    def claims_dataframe(self):
        ...
    
    @claims_dataframe.setter
    @abstractmethod
    def claims_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_claims_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_claims_metadata():
        ...

    @staticmethod
    @abstractmethod
    def claims_base_table(self):
        ...


@dataclass(frozen=True)
class ClaimsMetadata:
    string = TypeVar("string", bound=str)

    claim_no: int = field(metadata={"claim_no": "Claim Number (MONTHLY SEE FIELD NAME = CLAIM_NO)"})

    accept_stat_cd: string = field(metadata={"accept_stat_cd": "Indicates whether a claim was accepted or denied, A = ACCEPTED, ~ = MISSING, D = DENIED, P = PENDING"})

    enc_no: int = field(metadata={"enc_no": "The encounter number can be thought of as a billed healthcare event (by a single healthcare-providing entity)."})

    frp_seq_no: int = field(metadata={"frp_seq_no": "FRP_SEQ_NO indicates the order of payer responsibility on an encounter when HealthPartners is both the primary and secondary payer.  This most often happens when a member is covered under HealthPartners as their employer and then has a secondary coverage through their spouse under HealthPartners."})


class ClaimsData(MedicalClaims, ClaimsMetadata):
    CLAIMS_DF = spark.read.table('cleansed.adminsystemsclaims.claims')

    def __init__(self):
        self.__claims_df = None

    @property
    def claims_dataframe(self):
        return self.__claims_df
    
    @claims_dataframe.setter
    def claims_dataframe(self, dataframe):
        claims_df = dataframe.select(
            col("CLMNUM")\
                .cast(str(self.__dataclass_fields__["claim_no"].type.__name__))\
                    .alias("claim_no", metadata={"claim_no": self.__dataclass_fields__["claim_no"].metadata.get("claim_no")}), 
            col("ACCEPTANCESTATUS")\
                .cast(str(self.__dataclass_fields__["accept_stat_cd"].type.__name__))\
                    .alias("accept_stat_cd", metadata={"accept_stat_cd": self.__dataclass_fields__["accept_stat_cd"].metadata.get("accept_stat_cd")}), 
            col("ENCTRNUM")\
                .cast(str(self.__dataclass_fields__["enc_no"].type.__name__))\
                    .alias("enc_no", metadata={"enc_no": self.__dataclass_fields__["enc_no"].metadata.get("enc_no")}),
            col("FRPNUM")\
                .cast(str(self.__dataclass_fields__["frp_seq_no"].type.__name__))\
                    .alias("frp_seq_no", metadata={"frp_seq_no": self.__dataclass_fields__["frp_seq_no"].metadata.get("frp_seq_no")})
            )
        self.__claims_df = claims_df

    def create_claims_data_df(self):
        self.claims_dataframe = self.CLAIMS_DF
        _claims_dataframe = self.claims_dataframe
        _claims_dataframe.display()    
        return _claims_dataframe
    
    @staticmethod
    def view_claims_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(ClaimsMetadata)
            ]
        medical_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        medical_metadata_df.display()

    @staticmethod
    def claims_base_table():
        ClaimsData.CLAIMS_DF.display()
    
    def __repr__(self):
        return f'{self.__claims_df.display()}'

### Encounters Fields

#### Sourcing the Fields
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.encounters ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsclaims.encounters* 


| Source Field | Type Comment |
| :--- | :--- |
| clmvarclmformtype | Claim Varient Claim Form Type (MONTHLY SEE FIELD NAME = CLAIM_FORM_TP_CD) (Pointer to ^DIW-CLAIM_FORM_TYPES) |
| enctrnum | Encounter number generated by the production system. (MONTHLY SEE FIELD NAME = ENC_NO) Pointer to ^ENV-ENCOUNTERS) |
| fac | Facility (MONTHLY SEE FIELD NAME = FACIL_ID) (Pointer to ^DI00-FACILITIES) |
| firstsvdate | First Service Date (MONTHLY SEE FIELD NAME = FIRST_SERV_DT) |
| personnum | Person Number (MONTHLY SEE FIELD NAME = PERSON_NO) (Pointer to ^MB0-PERSONS) |


#### Accessing the Data
| Encounters Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| EncountersData( ).encounters_base_table( ) | EncountersData( ).create_encounters_data_df( ) | EncountersData( ).view_encounters_metadata( ) |


#### Development Comments
Summary
- Encounters medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the EncountersMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalEncounters(metaclass = ABCMeta):
    @property
    @abstractmethod
    def encounters_dataframe(self):
        ...
    
    @encounters_dataframe.setter
    @abstractmethod
    def encounters_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_encounters_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_encounters_metadata():
        ...

    @staticmethod
    @abstractmethod
    def encounters_base_table(self):
        ...


@dataclass(frozen=True)
class EncountersMetadata:
    string = TypeVar("string", bound=str)

    claim_form_tp_cd: string = field(metadata={"claim_form_tp_cd": "HCFA 1450 (facility)HCFA 1500 (professional)Narrative Claim (professional)"})

    enc_no: int = field(metadata={"enc_no": "The encounter number can be thought of as a billed healthcare event (by a single healthcare-providing entity)."})

    facil_id: int = field(metadata={"facil_id": "Internal Facility Identification Number.  Identifies the biling entity (facility) of the hospital, clinic or office at which the service was performed. * This is sometimes referred to as HPFIN (HealthPartners Facility Identification Number)To avoid record duplication, descriptive info was obtained from CLAIM_SER_DS.  If duplicates existsed here, the record with the most money in AbsoluteValue(TO_BE_PAID_AMT) and AbsolutValue(MEM_TOT_LIAB_AMT) was kept.If there are still duplicates, keep line with PREVENTIVE_SV_FLG = [blank] before PREVENTIVE_SV_FLG = YAt this point, if there are still duplicates, we let SAS randomly chose which record to keep because we do not know which one is better."})

    person_no: float = field(metadata={"person_no": "Internal person identification number"})
    

class EncountersData(MedicalEncounters, EncountersMetadata):
    ENCOUNTERS_DF = spark.read.table('cleansed.adminsystemsclaims.encounters')

    def __init__(self):
        self.__encounters_df = None

    @property
    def encounters_dataframe(self):
        return self.__encounters_df
    
    @encounters_dataframe.setter
    def encounters_dataframe(self, dataframe):
        encounters_df = dataframe.select(
            col("CLMVARCLMFORMTYPE")\
                .cast(str(self.__dataclass_fields__["claim_form_tp_cd"].type.__name__))\
                    .alias("claim_form_tp_cd", metadata={"claim_form_tp_cd": self.__dataclass_fields__["claim_form_tp_cd"].metadata.get("claim_form_tp_cd")}), 
            col("ENCTRNUM")\
                .cast(str(self.__dataclass_fields__["enc_no"].type.__name__))\
                    .alias("enc_no", metadata={"enc_no": self.__dataclass_fields__["enc_no"].metadata.get("enc_no")}),
            col("FAC")\
                .cast(str(self.__dataclass_fields__["facil_id"].type.__name__))\
                    .alias("facil_id", metadata={"facil_id": self.__dataclass_fields__["facil_id"].metadata.get("facil_id")}),
            col("PERSONNUM")\
                .cast(str(self.__dataclass_fields__["person_no"].type.__name__))\
                    .alias("person_no", metadata={"person_no": self.__dataclass_fields__["person_no"].metadata.get("person_no")}) 
            )
        self.__encounters_df = encounters_df

    def create_encounters_data_df(self):
        self.encounters_dataframe = self.ENCOUNTERS_DF
        _encounters_dataframe = self.encounters_dataframe  
        _encounters_dataframe.display()
        return _encounters_dataframe
    
    @staticmethod
    def view_encounters_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(EncountersMetadata)
            ]
        encounters_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        encounters_metadata_df.display()

    @staticmethod
    def encounters_base_table():
        EncountersData.ENCOUNTERS_DF.display()
    
    def __repr__(self):
        return f'{self.__encounters_df.display()}'

### EnctrFrpPackages Fields

#### Sourcing the Fields
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.enctrfrppackages ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsclaims.enctrfrppackages* 


| Source Field | Type Comment |
| :--- | :--- |
| enctrnum | Encounter number generated by the production system. (MONTHLY SEE FIELD NAME = ENC_NO) Pointer to ^ENV-ENCOUNTERS) |
| pkgnum | 'ben_pkg_id' comes from enctrfrppackages.pkgnum |
| product | Product  (MONTHLY SEE FIELD NAME = PROD_ID) (Pointer to ^DI1Y-PRODUCTS) |
| deliverynetwk | Delivery Network (Pointer to ^DIDA(1)-DELIVERY_NETWORK) |2

 
#### Accessing the Data
| EnctrFrpPackages Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| EnctrFrpPackagesData( ).enctrfrppackages_base_table( ) | EnctrFrpPackagesData( ).create_enctrfrppackages_data_df( ) | EnctrFrpPackagesData( ).view_enctrfrppackages_metadata( ) |


#### Development Comments
Summary
- EnctrFrpPackages medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the EnctrFrpPackagesMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalEnctrFrpPackages(metaclass = ABCMeta):
    @property
    @abstractmethod
    def enctrfrppackages_dataframe(self):
        ...
    
    @enctrfrppackages_dataframe.setter
    @abstractmethod
    def enctrfrppackages_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_enctrfrppackages_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_enctrfrppackages_metadata():
        ...

    @staticmethod
    @abstractmethod
    def enctrfrppackages_base_table(self):
        ...


@dataclass(frozen=True)
class EnctrFrpPackagesMetadata:
    string = TypeVar("string", bound=str)

    enc_no: int = field(metadata={"enc_no": "The encounter number can be thought of as a billed healthcare event (by a single healthcare-providing entity)."})

    delivery_network_id: int = field(metadata={"delivery_network_id": "Delivery Network Identifier that indicates the network of care the members coverage has available (i.e. OPEN ACCESS + CIGNA, ALLINA TIERED OPEN ACCESS, etc.)NOTE: To get discrete product groupings, sometimes you need to use PURCH_TP_NM, PURCH_SUB_TP_NM, PROD_NM and DELIVERY_NETWORK_NM.Look in ADW.DELIVERY_NETWORK_DS for dictionary of current values."})

    package_no: int = field(metadata={"package_no": "ben_pkg_id comes from enctrfrppackages.pkgnum"})

    prod_id: int = field(metadata={"prod_id": "Internal product identifier number. NOTE: To get discrete product groupings, sometimes you need to use PURCH_TP_NM, PURCH_SUB_TP_NM, PROD_NM and DELIVERY_NETWORK_NM."})
    

class EnctrFrpPackagesData(MedicalEnctrFrpPackages, EnctrFrpPackagesMetadata):
    ENCTRFRPPACKAGESDATA_DF = spark.read.table('cleansed.adminsystemsclaims.enctrfrppackages')

    def __init__(self):
        self.__enctrfrppackages_df = None

    @property
    def enctrfrppackages_dataframe(self):
        return self.__enctrfrppackages_df
    
    @enctrfrppackages_dataframe.setter
    def enctrfrppackages_dataframe(self, dataframe):
        enctrfrppackages_df = dataframe.select(
            col("ENCTRNUM")\
                .cast(str(self.__dataclass_fields__["enc_no"].type.__name__))\
                    .alias("enc_no", metadata={"enc_no": self.__dataclass_fields__["enc_no"].metadata.get("enc_no")}), 
            col("DeliveryNetwk")\
                .cast(str(self.__dataclass_fields__["delivery_network_id"].type.__name__))\
                    .alias("delivery_network_id", metadata={"delivery_network_id": self.__dataclass_fields__["delivery_network_id"].metadata.get("delivery_network_id")}), 
            col("PkgNum")\
                .cast(str(self.__dataclass_fields__["package_no"].type.__name__))\
                    .alias("package_no", metadata={"package_no": self.__dataclass_fields__["package_no"].metadata.get("package_no")}),
            col("Product")\
                .cast(str(self.__dataclass_fields__["prod_id"].type.__name__))\
                    .alias("prod_id", metadata={"prod_id": self.__dataclass_fields__["prod_id"].metadata.get("prod_id")})
            )
        self.__enctrfrppackages_df = enctrfrppackages_df

    def create_enctrfrppackages_data_df(self):
        self.enctrfrppackages_dataframe = self.ENCTRFRPPACKAGESDATA_DF
        _enctrfrppackages_dataframe = self.enctrfrppackages_dataframe  
        _enctrfrppackages_dataframe.display()
        return _enctrfrppackages_dataframe
    
    @staticmethod
    def view_enctrfrppackages_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(EnctrFrpPackagesMetadata)
            ]
        enctrfrppackages_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        enctrfrppackages_metadata_df.display()

    @staticmethod
    def enctrfrppackages_base_table():
        EnctrFrpPackagesData.ENCTRFRPPACKAGESDATA_DF.display()
    
    def __repr__(self):
        return f'{self.__enctrfrppackages_df.display()}'

### EnctrSvcs Fields

#### Sourcing the Fields
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.enctrsvcs ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsclaims.enctrsvcs* 


| Source Field | Type Comment |
| :--- | :--- |
| cpt4hcpcsgrpcpt4 | CPT4 HCPCS Group CPT4 (Pointer to ^DIg-CPT4S) |
| cpt4hcpcsgrpplaceofsvc | CPT4 HCPCS Group Place of Service (MONTHLY SEE FIELD NAME = PLACE_OF_SERV_CD) (Pointer to ^DI1K-PLACES_OF_SERVICE) |
| enctrnum | Encounter number generated by the production system. (MONTHLY SEE FIELD NAME = ENC_NO) Pointer to ^ENV-ENCOUNTERS) |


#### Accessing the Data
| EnctrSvcs Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| EnctrSvcsData( ).enctrsvcs_base_table( ) | EnctrSvcsData( ).create_enctrsvcs_data_df( ) | EnctrSvcsData( ).view_enctrsvcs_metadata( ) |


#### Development Comments
Summary
- EnctrSvcs medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the EnctrSvcsMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalEnctrSvcs(metaclass = ABCMeta):
    @property
    @abstractmethod
    def enctrsvcs_dataframe(self):
        ...
    
    @enctrsvcs_dataframe.setter
    @abstractmethod
    def enctrsvcs_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_enctrsvcs_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_enctrsvcs_metadata():
        ...

    @staticmethod
    @abstractmethod
    def enctrsvcs_base_table(self):
        ...


@dataclass(frozen=True)
class EnctrSvcsMetadata:
    string = TypeVar("string", bound=str)

    enc_no: int = field(metadata={"enc_no": "The encounter number can be thought of as a billed healthcare event (by a single healthcare-providing entity)."})

    place_of_serv_cd: string = field(metadata={"place_of_serv_cd": "Type of facility the service was performed atTo avoid record duplication, descriptive info was obtained from CLAIM_SER_DS.  If duplicates existsed here, the record with the most money in AbsoluteValue(TO_BE_PAID_AMT) and AbsolutValue(MEM_TOT_LIAB_AMT) was kept.If there are still duplicates, keep line with PREVENTIVE_SV_FLG = [blank] before PREVENTIVE_SV_FLG = YAt this point, if there are still duplicates, we let SAS randomly chose which record to keep because we do not know which one is better."})


class EnctrSvcsData(MedicalEnctrSvcs, EnctrSvcsMetadata):
    ENCTRSVCS_DF = spark.read.table('cleansed.adminsystemsclaims.enctrsvcs')

    def __init__(self):
        self.__enctrsvcs_df = None

    @property
    def enctrsvcs_dataframe(self):
        return self.__enctrsvcs_df
    
    @enctrsvcs_dataframe.setter
    def enctrsvcs_dataframe(self, dataframe):
        enctrsvcs_df = dataframe.select(
            col("CPT4HCPCSGRPPLACEOFSVC")\
                .cast(str(self.__dataclass_fields__["place_of_serv_cd"].type.__name__))\
                    .alias("place_of_serv_cd", metadata={"place_of_serv_cd": self.__dataclass_fields__["place_of_serv_cd"].metadata.get("place_of_serv_cd")}),
            col("ENCTRNUM")\
                .cast(str(self.__dataclass_fields__["enc_no"].type.__name__))\
                    .alias("enc_no", metadata={"enc_no": self.__dataclass_fields__["enc_no"].metadata.get("enc_no")})
            )
        self.__enctrsvcs_df = enctrsvcs_df

    def create_enctrsvcs_data_df(self):
        self.enctrsvcs_dataframe = self.ENCTRSVCS_DF
        _enctrsvcs_dataframe = self.enctrsvcs_dataframe
        _enctrsvcs_dataframe.display()    
        return _enctrsvcs_dataframe
    
    @staticmethod
    def view_enctrsvcs_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(EnctrSvcsMetadata)
            ]
        enctrsvcs_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        enctrsvcs_metadata_df.display()

    @staticmethod
    def enctrsvcs_base_table():
        EnctrSvcsData.ENCTRSVCS_DF.display()
    
    def __repr__(self):
        return f'{self.__enctrsvcs_df.display()}'

### Person Fields

#### Sourcing the Fields
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.person ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsmember.person* 


| Source Field | Type Comment |
| :--- | :--- |
| personnum | Person Number (MONTHLY SEE FIELD NAME = PERSON_NO) (Pointer to ^MB0-PERSONS) |


#### Accessing the Data
| Person Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| PersonData( ).person_base_table( ) | PersonData( ).create_person_data_df( ) | PersonData( ).view_person_metadata( ) |


#### Development Comments
Summary
- Person medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the PersonMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalPerson(metaclass = ABCMeta):
    @property
    @abstractmethod
    def person_dataframe(self):
        ...
    
    @person_dataframe.setter
    @abstractmethod
    def person_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_person_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_person_metadata():
        ...

    @staticmethod
    @abstractmethod
    def person_base_table(self):
        ...


@dataclass(frozen=True)
class PersonMetadata:
    string = TypeVar("string", bound=str)

    external_person_id: int = field(metadata={"external_person_id": "External Identification Number.  This is sometimes referred to as member number or chart number.  This is the number you will see on your insurance card."})

    person_no: float = field(metadata={"person_no": "Internal person identification number"})


class PersonData(MedicalPerson, PersonMetadata):
    PERSON_DF = spark.read.table('cleansed.adminsystemsmember.person')

    def __init__(self):
        self.__person_df = None

    @property
    def person_dataframe(self):
        return self.__person_df
    
    @person_dataframe.setter
    def person_dataframe(self, dataframe):
        person_df = dataframe.select(
            col("DemographicsExternalIdNum")\
                .cast(str(self.__dataclass_fields__["external_person_id"].type.__name__))\
                    .alias("external_person_id", metadata={"external_person_id": self.__dataclass_fields__["external_person_id"].metadata.get("external_person_id")}),
            col("PersonNum")\
                .cast(str(self.__dataclass_fields__["person_no"].type.__name__))\
                    .alias("person_no", metadata={"person_no": self.__dataclass_fields__["person_no"].metadata.get("person_no")}), 
            )
        self.__person_df = person_df

    def create_person_data_df(self):
        self.person_dataframe = self.PERSON_DF
        _person_dataframe = self.person_dataframe  
        _person_dataframe.display()
        return _person_dataframe
    
    @staticmethod
    def view_person_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(PersonMetadata)
            ]
        person_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        person_metadata_df.display()

    @staticmethod
    def person_base_table():
        PersonData.PERSON_DF.display()
    
    def __repr__(self):
        return f'{self.__person_df.display()}'

### Products Fields

#### Sourcing the Fields
Summary
- All table fields have been pulled from Adminsystems ( table : cdcods.products ) and have been ingested into the cleansed production cloud environment location: *cleansed.adminsystemsdict.products* 


| Source Field | Type Comment |
| :--- | :--- |
| product | Product  (MONTHLY SEE FIELD NAME = PROD_ID) (Pointer to ^DI1Y-PRODUCTS) |
| purchasertype | Purchaser Subtype, Purchaser Subtype (1 of 2 col; Pointer to ^MB82-PRODUCT_PURCHASER_SUBTYPES) |
| purchasersubtypepurchsubtype | Purchaser Subtype, Purchaser Type (2 of 2 col; Pointer to ^MB82-PRODUCT_PURCHASER_SUBTYPES) |


#### Accessing the Data
| Products Table | Medical Fields Dataframe | Medical Fields Metadata |
| :--- | :--- | :--- |
| ProductsData( ).products_base_table( ) | ProductsData( ).create_products_data_df( ) | ProductsData( ).view_products_metadata( ) |


#### Development Comments
Summary
- Products medical fields dataframe column types are set using pyspark.sql.Column.**cast**. Assigning the column type within the ProductsMetadata dataclass accomplishes the casting. ( field : type )

In [ ]:
class MedicalProducts(metaclass = ABCMeta):
    @property
    @abstractmethod
    def products_dataframe(self):
        ...
    
    @products_dataframe.setter
    @abstractmethod
    def products_dataframe(self, dataframe):
        ...

    @abstractmethod
    def create_products_data_df(self):
        ...

    @staticmethod
    @abstractmethod
    def view_products_metadata():
        ...

    @staticmethod
    @abstractmethod
    def products_base_table(self):
        ...


@dataclass(frozen=True)
class ProductsMetadata:
    string = TypeVar("string", bound=str)

    prod_id: int = field(metadata={"prod_id": "Internal product identifier number. NOTE: To get discrete product groupings, sometimes you need to use PURCH_TP_NM, PURCH_SUB_TP_NM, PROD_NM and DELIVERY_NETWORK_NM."})

    purch_sub_tp_nm: string = field(metadata={"purch_sub_tp_nm": "Product Purchaser Subtype: Conversion, Cost Group, Cost Individual, Individual, Large Group M+C Group, M+C Individual, PMAP, GAMC, MnCare, Dual Eligible, Select Individual, Small Group, Supplemental IndividualNOTE: To get discrete product groupings, sometimes you need to use PURCH_TP_NM, PURCH_SUB_TP_NM, PROD_NM and DELIVERY_NETWORK_NM.HPUPH Patchif grp_no = 32116 then purch_sub_tp_nm = GROUP - HPUPH; else; Senior Retiree Ntional Choice Patchif prod_id in (848, 855) then PURCH_SUB_TP_NM = GROUP - RNC; else; Note, if people need to find out the original values, they could link back to PROD_DIM by PROD_ID."})

    purch_tp_nm: string = field(metadata={"purch_tp_nm": "productPurchaser Type full name: Commercial, Medicaid, or Medicare based on product. If SUBSIDIZED INDIVIDUAL EXCHANGE then COMMERCIAL else purch_tp_nmNOTE: To get discrete product groupings, sometimes you need to use PURCH_TP_NM, PURCH_SUB_TP_NM, PROD_NM and DELIVERY_NETWORK_NM.Since members can be enrolled in more than one product we recommend always stratifying analysis by product to prevent double counting.  For example, if a members is enrolled in both Medicare and Commercial products and we administer both plans we will get two bills for the service but the service did not occur twice."})

    
class ProductsData(MedicalProducts, ProductsMetadata):
    PRODUCTS_DF = spark.read.table('cleansed.adminsystemsdict.products')

    def __init__(self):
        self.__products_df = None

    @property
    def products_dataframe(self):
        return self.__products_df
    
    @products_dataframe.setter
    def products_dataframe(self, dataframe):
        products_df = dataframe.select(
            col("PurchaserSubtypePurchaserSubtype")\
                .cast(str(self.__dataclass_fields__["purch_sub_tp_nm"].type.__name__))\
                    .alias("purch_sub_tp_nm", metadata={"purch_sub_tp_nm": self.__dataclass_fields__["purch_sub_tp_nm"].metadata.get("purch_sub_tp_nm")}), 
            col("PurchaserType")\
                .cast(str(self.__dataclass_fields__["purch_tp_nm"].type.__name__))\
                    .alias("purch_tp_nm", metadata={"purch_tp_nm": self.__dataclass_fields__["purch_tp_nm"].metadata.get("purch_tp_nm")}),
            col("ProductNum")\
                .cast(str(self.__dataclass_fields__["prod_id"].type.__name__))\
                    .alias("prod_id", metadata={"prod_id": self.__dataclass_fields__["prod_id"].metadata.get("prod_id")}), 
            )
        self.__products_df = products_df

    def create_products_data_df(self):
        self.products_dataframe = self.PRODUCTS_DF
        _products_dataframe = self.products_dataframe 
        _products_dataframe.display()  
        return _products_dataframe
    
    @staticmethod
    def view_products_metadata():
        metadata = [[
            field.name, 
            field.type.__name__, 
            field.metadata.get(field.name)] for field in fields(ProductsMetadata)
            ]
        products_metadata_df = spark.createDataFrame(
            metadata, 
            schema="Name STRING, Type STRING, Comments STRING"
            )  
        products_metadata_df.display()

    @staticmethod
    def products_base_table():
        ProductsData.PRODUCTS_DF.display()
    
    def __repr__(self):
        return f'{self.__products_df.display()}'

### __Hicore Medical DataFrame


#### Dataframe Details
| Name | Keys (Primary, Surrogate) | 
| --- | --- | 
| claims_df | Surrogate key: 'enc_no' | 
| encounters_df | Surrogate keys: 'enc_no', 'person_id' | 
| enctrfrppackages_df | Surrogate keys: 'enc_no', 'prod_id' | 
| person_df | Surrogate key: 'external_person_id' | 
| products_df | Surrogate key: 'prod_id' | 


#### Development Comments
Summary
- The Hicore Medical dataframe is aggregated when 'HicoreMedicalDataFrame' is instantiated to a new object. Example: new_dataframe = HicoreMedicalDataFrame( ). The complete dataframe is returned with *new_dataframe.hicore_medical_df*. 
- Hicore Medical table logic occurs in post initialization (__post_init__) of the dataclass.

In [ ]:
@dataclass
class HicoreMedicalDataFrame:
    hicore_medical_df: ps.DataFrame = field(default=None)
    adjudicatedsvcs_df: ps.DataFrame = field(default=AdjudicatedSvcsData().create_adjudicatedsvcs_data_df(), \
        metadata={"adjudicatedsvcs_df": "Surrogate key: 'claim_no'. Shared relationship with 'Claims' dataframe"})
    benefitpkg_df: ps.DataFrame = field(default=BenefitPkgData().create_benefitpkg_data_df(), \
        metadata={"benefitpkg_df": "Surrogate key: 'ben_pkg_cd'. Shared relationship with 'Enctrfrppackages' dataframe"})
    claims_df: ps.DataFrame = field(default=ClaimsData().create_claims_data_df(), \
        metadata={"claims_df": "Surrogate key: 'enc_no'. Shared relationship with 'Encounters' dataframe"})
    encounters_df: ps.DataFrame = field(default=EncountersData( ).create_encounters_data_df(), \
        metadata={"ecounters_df": "Surrogate key: 'enc_no'. Shared relationship with 'Claims' and 'Enctrfrppackages' dataframes. Surrogate key: 'person_no'. Shared relationship with 'Person' dataframe"})
    enctrsvcs_df: ps.DataFrame = field(default=EnctrSvcsData().create_enctrsvcs_data_df(), \
        metadata={"claims_df": "Surrogate key: 'enc_no'. Shared relationship with 'Enctrfrppackages' dataframe"})
    enctrfrppackages_df: ps.DataFrame = field(default=EnctrFrpPackagesData().create_enctrfrppackages_data_df(), \
        metadata={"enctrfrppackages_df": "Surrogate key: 'enc_no'. Shared relationship with 'Encounters' dataframe. Surrogate key: 'prod_id'. Shared relationship with 'Products' dataframe"}) 
    person_df: ps.DataFrame = field(default=PersonData().create_person_data_df(), \
        metadata={"person_df": "Surrogate key: 'external_person_id'. Shared relationship with 'Encounters' dataframe."}) 
    products_df: ps.DataFrame = field(default=ProductsData().create_products_data_df(), \
        metadata={"products_df": "Surrogate key: 'prod_id'. Shared relationship with 'Enctrfrppackages' dataframe."}) 
    

    def __post_init__(self):
        self.hicore_medical_df = self.claims_df \
            .join(self.adjudicatedsvcs_df, "claim_no") \
                .join(self.encounters_df, "enc_no") \
                    .join(self.enctrfrppackages_df, "enc_no") \
                        .join(self.enctrsvcs_df, "enc_no") \
                            .join(self.products_df, "prod_id") \
                                .join(self.benefitpkg_df, "package_no") \
                                    .join(self.person_df, "person_no") \

    @property
    def transformed_dataframe(self):
        return self.hicore_medical_df


dataframe = HicoreMedicalDataFrame()
hicore_medical_dataframe = dataframe.hicore_medical_df.display()

### @Hicore Medical Delta Live Table

#### --Materialized View--

In [ ]:
@dlt.table(
    name='hicore_medical_test',
    comment='CLAIM_SER_PYMT_IS and other claims/coverage data in a single location'
)
def hicore_medical_delta_live():
    dataframe = HicoreMedicalDataFrame()
    hicore_medical_dataframe = dataframe.hicore_medical_df
    return hicore_medical_dataframe

### @Hicore Medical Table

#### --Managed Table--

In [ ]:
table_location = {
    'catalog_name': 'standardized_dev',
    'schema_name': 'adwhicore',
    'table_name': 'hicore_medical_base'
}

def hicore_medical_table():
    dataframe = HicoreMedicalDataFrame()
    hicore_medical_dataframe = dataframe.hicore_medical_df
    
    hicore_medical_dataframe \
        .write.format('delta') \
            .mode('overwrite') \
                .saveAsTable( \
                    f'{table_location["catalog_name"]} \
                        .{table_location["schema_name"]} \
                            .{table_location["table_name"]}')

hicore_medical_table()